# 基于人工智能的干旱趋势预测研究

在开始前，请运行下面这行代码以确保您的环境运行正常：

**特别注意：请按照从上至下的顺序依次运行每个代码框中的内容，不要跳跃！不要跳跃！！不要跳跃！！！**

In [1]:
print("Hello World!")

Hello World!


以下代码将为您导入需要的库与模块。需要注意的是，在运行前您需要在终端中输入`pip list`或`conda list`以检查您的环境中是否已包含下列库：

In [2]:
print("Loading libs...")

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import math
import netCDF4 as nc
import time
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation
from tensorflow.keras.layers import Dropout, LSTM
from sklearn.metrics import mean_squared_error, mean_absolute_error

print("All the libs are loaded.")

Loading libs...
All the libs are loaded.


我们把程序运行中所有可能需要更改的参数全部集中于此，您可以方便地在此处集中更改：

In [3]:
nc_filepath1 = "/home/featurize/data/(1959-1990)adaptor.mars.internal-1660707411.5805993-7409-10-0c781592-41e4-4118-aa52-45d0b9c7a1c2.nc"
nc_filepath2 = "/home/featurize/data/(1991-2021)adaptor.mars.internal-1660823200.6822143-31922-11-67b5e941-766f-4f4c-bfe0-93bf41f48824.nc"
SPI3_filepath = "/home/featurize/work/spi3(1979.1-2021.6).txt"
nc_file1 = nc.Dataset(nc_filepath1)
nc_file2 = nc.Dataset(nc_filepath2)
SPI3_file = open(SPI3_filepath, encoding='UTF-8')

定义一个2D ResNet18网络，此部分代码来自北京大学Tensorflow 2.0教程中老师的课堂演示。在<code>class ResNet18</code>的末尾更改<code>y</code>的形状是为了将其与LSTM可以接收的维度相匹配：

In [4]:
print("Defining ResNet18...")

class ResnetBlock(Model):

    def __init__(self, filters, shape, strides=1, residual_path=False):
        super(ResnetBlock, self).__init__()
        self.filters = filters
        self.strides = strides
        self.residual_path = residual_path

        self.c1 = Conv2D(filters, (3, 3), strides=strides, padding='same', use_bias=False, input_shape=shape)
        self.b1 = BatchNormalization()
        self.a1 = Activation('relu')

        self.c2 = Conv2D(filters, (3, 3), strides=1, padding='same', use_bias=False, input_shape=shape)
        self.b2 = BatchNormalization()

        # residual_path为True时，对输入进行下采样，即用1x1的卷积核做卷积操作，保证x能和F(x)维度相同，顺利相加
        if residual_path:
            self.down_c1 = Conv2D(filters, (1, 1), strides=strides, padding='same', use_bias=False, input_shape=shape)
            self.down_b1 = BatchNormalization()
        
        self.a2 = Activation('relu')

    def call(self, inputs):
        residual = inputs  # residual等于输入值本身，即residual=x
        # 将输入通过卷积、BN层、激活层，计算F(x)
        x = self.c1(inputs)
        x = self.b1(x)
        x = self.a1(x)

        x = self.c2(x)
        y = self.b2(x)

        if self.residual_path:
            residual = self.down_c1(inputs)
            residual = self.down_b1(residual)

        out = self.a2(y + residual)  # 最后输出的是两部分的和，即F(x)+x或F(x)+Wx,再过激活函数
        return out


class ResNet18(Model):

    def __init__(self, block_list, shape, initial_filters=64):  # block_list表示每个block有几个卷积层
        super(ResNet18, self).__init__()
        self.num_blocks = len(block_list)  # 共有几个block
        self.block_list = block_list
        self.shape = shape
        self.out_filters = initial_filters
        self.c1 = Conv2D(self.out_filters, kernel_size=3, strides=1, padding='same', use_bias=False, input_shape = shape[1:])
        self.b1 = BatchNormalization()
        self.a1 = Activation('relu')
        self.blocks = tf.keras.models.Sequential()
        # 构建ResNet网络结构
        for block_id in range(len(block_list)):  # 第几个resnet block
            for layer_id in range(block_list[block_id]):  # 第几个卷积层

                if block_id != 0 and layer_id == 0:  # 对除第一个block以外的每个block的输入进行下采样
                    block = ResnetBlock(self.out_filters, shape, strides=2, residual_path=True)
                else:
                    block = ResnetBlock(self.out_filters, shape, residual_path=False)
                self.blocks.add(block)  # 将构建好的block加入resnet
            self.out_filters *= 2  # 下一个block的卷积核数是上一个block的2倍
        # self.p1 = GlobalAveragePooling2D()
        #self.f1 = Dense(10, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2())

    def call(self, inputs):
        #print("resnet:",inputs.shape)
        x = self.c1(inputs)
        x = self.b1(x)
        x = self.a1(x)
        x = self.blocks(x)
        #print("resnest:",x.shape)
        # y = self.p1(x)
        y = tf.reshape(x,(batch_size,-1,x.shape[-1]))   # 这句话改过了
        #print("resnet:",y.shape)
        #y = self.f1(x)
        return y

print("ResNet18 have been successfully defined.")

Defining ResNet18...
ResNet18 have been successfully defined.


导入nc文件数据：

In [5]:
x = []
temp = []
keylist = list(nc_file1.variables.keys())

# monthnum_file1 = 32*12, 32 yrs & 12 months-per-yr
# monthnum_file2 = 31*12, 31 yrs & 12 months-per-yr
for key in keylist[4:5]:                             # 忽略前四个变量名是因为前四个变量名分别对应经度、纬度、高度、时间
    loadingStart = time.perf_counter()
    for month in nc_file1.variables[key][243:]:     #从1979.1开始
        # shape(month) = (4,721,1440), 3D
        # temp1 = np.reshape(month,(-1,1440)); shape(temp1) = (2884,1440), 2D
        temp.append(np.reshape(month,(-1,1440)))    # shape(temp) = (monthnum_file1,2884,1440), 3D
    for month in nc_file2.variables[key][:-6]:      #至2021.6结束，共498个月
        # shape(month) = (4,721,1440)
        # temp1 = np.reshape(month,(-1,1440)); shape(temp1) = (2884,1440), 2D
        temp.append(np.reshape(month,(-1,1440)))    # shape(temp) = (monthnum_file1+monthnum_file2,2884,1440), 3D
    x.append(temp)                                  # shape(x) = (16,monthnum_file1+monthnum_file2,2884,1440), 4D
    temp = []
    loadingEnd = time.perf_counter()
    loadingTime = round(loadingEnd-loadingStart, 3)
    print("Successfully appended variable {} within {} secs.".format(key, loadingTime))

x = np.transpose(x, (1,3,2,0))                      # shape(x) = (monthnum_file1+monthnum_file2,16,2884,1440), 4D
train_test_sep = 400
x_train = np.array(x[:train_test_sep])
x_test = np.array(x[train_test_sep:])
print("*********The ERA5 dataset has been successfully loaded.*********")

Successfully appended variable d within 31.386 secs.
*********The ERA5 dataset has been successfully loaded.*********


加载SPI3数据：

In [6]:
str_lst = SPI3_file.readlines()
y = [float(x) for x in str_lst]
SPI3_file.close()

y_train = np.array(y[3:train_test_sep+3])
y_test = np.array(y[train_test_sep+3:])

模型训练：

In [ ]:
batch_size = 2
CNN = ResNet18([2, 2, 2, 2], (batch_size, 1440, 2884, 16))
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
with strategy.scope():
    model = tf.keras.models.Sequential([
        CNN,
        # tf.keras.layers.TimeDistributed(CNN),    # 将CNN网络打包在TD层中实现复用
        LSTM(50),
        Dropout(0.1),
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                loss='mean_squared_error') 

checkpoint_save_path = "./checkpoint/LRCN.ckpt"
if os.path.exists(checkpoint_save_path + '.index'):
    print('-------------load the model-----------------')
    model.load_weights(checkpoint_save_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,
                                                 save_weights_only=True,
                                                 save_best_only=True,
                                                 monitor='val_loss')
print("pre_fit:",x_train.shape)
print("pre_fit:",x_test.shape)

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=50, validation_data=(x_test, y_test), 
                    validation_freq=1, callbacks=[cp_callback])

model.summary()

2022-08-29 17:18:21.966477: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2022-08-29 17:18:21.966871: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-29 17:18:21.975331: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-29 17:18:21.975694: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-29 17:18:21.975966: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-29 17:18:21.976225: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

Number of devices: 2
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
pre_fit: (400, 1440, 2884, 1)
pre_fit: (107, 1440, 2884, 1)


2022-08-29 17:18:31.100250: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 6644736000 exceeds 10% of free system memory.
2022-08-29 17:18:39.593934: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 6644736000 exceeds 10% of free system memory.
2022-08-29 17:18:46.223745: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 6644736000 exceeds 10% of free system memory.
2022-08-29 17:18:52.762235: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batc

Epoch 1/50
INFO:tensorflow:batch_all_reduce: 63 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:batch_all_reduce: 63 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


2022-08-29 17:19:04.186274: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101
2022-08-29 17:19:05.709405: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101
2022-08-29 17:19:09.563203: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 525.13MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-08-29 17:19:09.647078: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.13GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-08-29 17:19:09.647146: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.13GiB with freed_by_count=0. The cal

In [ ]:
file = open('./weights.txt', 'w')  # 参数提取
for v in model.trainable_variables:
    file.write(str(v.name) + '\n')
    file.write(str(v.shape) + '\n')
    file.write(str(v.numpy()) + '\n')
file.close()

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
################## predict ######################
# 测试集输入模型进行预测
predicted_SPI3 = model.predict(x_test)
"""# 对预测数据还原---从（0，1）反归一化到原始范围
predicted_stock_price = sc.inverse_transform(predicted_stock_price)
# 对真实数据还原---从（0，1）反归一化到原始范围
real_stock_price = sc.inverse_transform(test_set[60:])"""
# 画出真实数据和预测数据的对比曲线
plt.plot(y_test, color='red', label='SPI3')
plt.plot(predicted_SPI3, color='blue', label='Predicted SPI3')
plt.title('SPI3 Prediction')
plt.xlabel('Time')
plt.ylabel('SPI3')
plt.legend()
plt.show()

In [ ]:
##########evaluate##############
# calculate MSE 均方误差 ---> E[(预测值-真实值)^2] (预测值减真实值求平方后求均值)
mse = mean_squared_error(predicted_SPI3, y_test)
# calculate RMSE 均方根误差--->sqrt[MSE]    (对均方误差开方)
rmse = math.sqrt(mean_squared_error(predicted_SPI3, y_test))
# calculate MAE 平均绝对误差----->E[|预测值-真实值|](预测值减真实值求绝对值后求均值）
mae = mean_absolute_error(predicted_SPI3, y_test)
print('均方误差: %.6f' % mse)
print('均方根误差: %.6f' % rmse)
print('平均绝对误差: %.6f' % mae)